## Loading the Dataset

In [201]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import json
from web3 import Web3
# df['target']                             

In [202]:
ganache_url = "http://127.0.0.1:8545"
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.isConnected()

True

In [203]:
abi_train = json.loads('[ { "inputs": [ { "internalType": "int256[]", "name": "feature", "type": "int256[]" }, { "internalType": "int256", "name": "n", "type": "int256" } ], "name": "get_mean", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "int256[]", "name": "feature", "type": "int256[]" }, { "internalType": "int256", "name": "m", "type": "int256" }, { "internalType": "int256", "name": "n", "type": "int256" } ], "name": "get_var", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [], "name": "set_mean", "outputs": [ { "internalType": "int256", "name": "", "type": "int256" } ], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [], "name": "set_var", "outputs": [ { "internalType": "int256", "name": "", "type": "int256" } ], "stateMutability": "nonpayable", "type": "function" } ]')
cont_t = "0x2488a09f3E87E61f8Daa9a5923903aD064e8e474"
cont_train = web3.eth.contract(address=cont_t,abi=abi_train)
abi_pred = json.loads('[ { "constant": false, "inputs": [ { "name": "x", "type": "int256[]" }, { "name": "v", "type": "int256[]" }, { "name": "m", "type": "int256[]" } ], "name": "a_getparameters1", "outputs": [], "payable": false, "stateMutability": "nonpayable", "type": "function" }, { "constant": false, "inputs": [ { "name": "e0", "type": "int256" }, { "name": "e1", "type": "int256" }, { "name": "f0", "type": "int256" }, { "name": "f1", "type": "int256" }, { "name": "g0", "type": "int256" }, { "name": "g1", "type": "int256" }, { "name": "h0", "type": "int256" }, { "name": "h1", "type": "int256" }, { "name": "Prior0", "type": "int256" }, { "name": "Prior1", "type": "int256" } ], "name": "e_exponent", "outputs": [], "payable": false, "stateMutability": "nonpayable", "type": "function" }, { "constant": true, "inputs": [], "name": "set_exponent", "outputs": [ { "name": "", "type": "int256" }, { "name": "", "type": "int256" } ], "payable": false, "stateMutability": "view", "type": "function" }, { "constant": true, "inputs": [], "name": "a_setparameters1", "outputs": [ { "name": "", "type": "int256" }, { "name": "", "type": "int256" }, { "name": "", "type": "int256" }, { "name": "", "type": "int256" } ], "payable": false, "stateMutability": "view", "type": "function" } ]')
cont_p = "0xCd70A8C7Ead60dF9575f0adD681f5c41163eD455"
cont_pred = web3.eth.contract(address=cont_p,abi=abi_pred)

In [204]:
#grit
account1 = "0xdd617f9D5120b7b30E4C502fdd6b13dCf286A300"
private_key1="c00b36dc1777e7fda60ff1f55b57c3f51b18c0c16fe44724e451ed062a173d74"
account2 = "0x8883f1016E727C8fE93C1Df471f102bBbAF2B910"
private_key2="8d70f54549b5fb7c04fc016cc88c447fc81aff3af3065c696aee1b529fbc8ba5"

In [205]:
def get_mean(feature, n, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx = cont_train.functions.get_mean(
        feature,n
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('10','gwei'),
    'from': account,
    'nonce': nonce
    })
    signed_tx = web3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

def get_variance(feature, mean, n, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx = cont_train.functions.get_var(
        feature,mean,n
    ).buildTransaction({
    'gas': 6000000,
    'gasPrice': web3.toWei('10','gwei'),
    'from': account,
    'nonce': nonce
    })
    signed_tx = web3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    
def get_class(features,means, variances, account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx = cont_pred.functions.a_getparameters1(
        features,means,variances
    ).buildTransaction({
    'gas': 6000000,
    'gasPrice': web3.toWei('10','gwei'),
    'from': account,
    'nonce': nonce
    })
    signed_tx = web3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

def get_probability(A0,A1,B0,B1,C0,C1,D0,D1,Prior0,Prior1,account, private_key,contract):
    nonce = web3.eth.getTransactionCount(account)
    tx = cont_pred.functions.e_exponent(
        A0,A1,B0,B1,C0,C1,D0,D1,Prior0,Prior1
    ).buildTransaction({
    'gas': 6000000,
    'gasPrice': web3.toWei('10','gwei'),
    'from': account,
    'nonce': nonce
    })
    signed_tx = web3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

def compare(A,B):
    if(A>B): return 0
    else: return 1

In [206]:
def mux1(v,k):
    x = 1
#     print(type(v))
    for i in range(len(v)):
        if(i==k): continue
        else:  x = x* v[i]
    return x

def compute_param(x,m,v):
    B0=1; C0 =1.0;  A0=1; e = 2.718
    c = np.zeros(len(v));
    for i in range(len(v)):
        B0 = B0* int(v[i])
        c[i] = (x[i]-m[i])*(x[i]-m[i])
    D0 = B0
    for i in range(len(c)):
        C0 = C0+c[i]*mux1(v,i) 
    ex = float(C0/D0)
    return (A0,B0,C0,D0)
      
def exponent(x, m1,m2, v1,v2, Prior0,Prior1):
        A0,B0,C0,D0 = compute_param(x,m1,v1)
        print(A0,B0,C0,D0)
        A1,B1,C1,D1 = compute_param(x,m2,v2)
        print(A1,B1,C1,D1)
        A = A0*B1
        B = B0*A1   
        C = (C0*D1)-(C1*D0)
        D = D0*D1

        Q = int(C/D)
        R = C%D;
       
        Enum=19
        Eden=7
        
        edenR = 24*(D*D*D*D)
        enumR = 24*D*D*D*D+ R*24*D*D*D+R*R*12*D*D+R*R*R*4*D+R*R*R*R
        
        if(Q>=0):
            P0 = A*1*edenR*Prior0*Prior0
            P1 = B*1000*enumR*Prior1*Prior1
        else:
            P0 = A*1000*edenR*Prior0*Prior0
            P1 = B*1*enumR*Prior1*Prior1
        if(P0>P1): 
            return 0
        else : 
            return 1


In [207]:
cont_train.functions.set_mean().call()
cont_train.functions.set_var().call()

5632

In [208]:
df = pd.read_csv("heart.csv")
df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [209]:
# df['oldpeak'] - df['oldpeak']*10
target = np.array(df['target'].values)
target.shape

(303,)

In [210]:
features = np.array(df.drop(columns = ['target']))
features.shape

(303, 13)

In [211]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [212]:
x_train,x_test, y_train, y_test = train_test_split(features,target,test_size=0.2, random_state=0)
x_train = (StandardScaler().fit_transform(x_train)*100).astype(int)
x_test = (StandardScaler().fit_transform(x_test)*100).astype(int)
# x_test

In [213]:
x_train.shape

(242, 13)

In [250]:
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)
print("Number of mislabeled points out of a total %d points : %d" % (x_test.shape[0], (y_test != y_pred).sum()))
print(accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 61 points : 12
0.8032786885245902


In [251]:
df = pd.read_csv("heart.csv")
target = np.array(df['target'].values)
features = np.array(df.drop(columns = ['target']))
dfp = df.groupby('target').size()
Prior0 = int(dfp[0]/len(df)*100)
Prior1 = int(dfp[1]/len(df)*100)

In [252]:
x_train,x_test, y_train, y_test = train_test_split(features,target,test_size=0.2, random_state=0)
x_train = (StandardScaler().fit_transform(x_train)*100).astype(int)
x_test = (StandardScaler().fit_transform(x_test)*100).astype(int)

In [253]:
df_new = pd.DataFrame(x_train,columns=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13'])
df_new['target'] = y_train
df_mean = df_new.groupby(['target']).mean().astype(int)
df_var = df_new.groupby(['target']).var().astype(int)
mean = df_mean.values
var = df_var.values

In [254]:
df_mean

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
target,,,,,,,,,,,,,
0,22,34,-46,15,6,6,-7,-44,47,44,-32,46,33
1,-19,-29,39,-12,-5,-4,7,37,-39,-37,27,-39,-27


In [255]:
df_var

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
target,,,,,,,,,,,,,
0,7326,6161,7549,11347,8532,11223,10726,9655,10972,12593,8538,10726,13499
1,11453,11268,8705,8506,11197,8921,9236,7239,5510,4669,9552,5939,5388


In [256]:
x_train_0 = df_new.loc[df_new['target']==0]
x_train_1 = df_new.loc[df_new['target']==1]
x_train_0_ar = x_train_0.to_numpy()
x_train_1_ar = x_train_1.to_numpy()

In [257]:
x_train_0_ar

array([[  91, -143,  -92, ...,  -70,  -46,    0],
       [  24,   69,    3, ...,   26,  -46,    0],
       [  46,   69,  194, ...,  123,  -46,    0],
       ...,
       [-132,   69,  -92, ...,  318,  114,    0],
       [  -9,   69,  -92, ...,   26,  114,    0],
       [  35,   69,    3, ...,  -70,  -46,    0]])

In [258]:
x_train_1.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,target
0,-132,-143,98,-57,-63,-41,90,65,-70,-72,-66,-70,-46,1
1,124,-143,98,83,58,-41,-97,9,-70,-89,-66,26,-46,1
2,35,69,98,47,-67,239,-97,65,-70,-89,95,-70,-46,1
7,-87,69,-92,-115,-80,-41,90,-29,-70,-80,95,-70,-46,1
8,-43,69,-92,47,26,-41,-97,156,140,-89,95,-70,-46,1


In [259]:
mean

array([[ 22,  34, -46,  15,   6,   6,  -7, -44,  47,  44, -32,  46,  33],
       [-19, -29,  39, -12,  -5,  -4,   7,  37, -39, -37,  27, -39, -27]])

In [260]:
var

array([[ 7326,  6161,  7549, 11347,  8532, 11223, 10726,  9655, 10972,
        12593,  8538, 10726, 13499],
       [11453, 11268,  8705,  8506, 11197,  8921,  9236,  7239,  5510,
         4669,  9552,  5939,  5388]])

In [261]:

_means0=[]
_means1=[]
for i in range(x_train.shape[1]):
    get_mean(x_train_0_ar[:,i].tolist(), x_train_0.shape[0], account1, private_key1,cont_train)
    _means0.append(cont_train.functions.set_mean().call())
    get_mean(x_train_1_ar[:,i].tolist(), x_train_1.shape[0], account1, private_key1,cont_train)
    _means1.append(cont_train.functions.set_mean().call())

In [262]:
print(_means0)
print(_means1)

[22, 34, -46, 15, 6, 6, -7, -44, 47, 44, -32, 46, 33]
[-19, -29, 39, -12, -5, -4, 7, 37, -39, -37, 27, -39, -27]


In [263]:
_var0=[]
_var1=[]
for i in range(x_train.shape[1]):        
    get_variance(x_train_0_ar[:,i].tolist(),int(_means0[i]), x_train_0.shape[0], account1, private_key1,cont_train)
    _var0.append(cont_train.functions.set_var().call())
    get_variance(x_train_1_ar[:,i].tolist(),int(_means1[i]), x_train_1.shape[0], account1, private_key1,cont_train)
    _var1.append(cont_train.functions.set_var().call())

In [264]:
print(_var0)
print(_var1)

[7260, 6106, 7481, 11245, 8455, 11123, 10630, 9568, 10873, 12480, 8461, 10629, 13377]
[11365, 11183, 8638, 8442, 11112, 8854, 9165, 7183, 5468, 4634, 9479, 5894, 5348]


In [265]:
# get_class1(features,means, variances, account, private_key,contract)
# get_class2(features,means,variances, account, private_key,contract)
# get_probability(A0,A1,B0,B1,C0,C1,D0,D1,Prior0,Prior1, private_key,contract)

In [266]:
print(x_test[i,:].tolist())
print(_means0)
print(_var0)

[134, 62, -101, 21, 21, -41, -112, -100, -64, 182, -59, 26, 106]
[22, 34, -46, 15, 6, 6, -7, -44, 47, 44, -32, 46, 33]
[7260, 6106, 7481, 11245, 8455, 11123, 10630, 9568, 10873, 12480, 8461, 10629, 13377]


In [267]:
# dfp = df.groupby('target').size()
Prior0 = int(len(x_train_0)/len(df_new)*100)
Prior1 = int(len(x_train_1)/len(df_new)*100)

print(Prior0, Prior1)

45 54


In [268]:
var0_less = np.sqrt(np.array(_var0)).astype(int)
var1_less = np.sqrt(np.array(_var1)).astype(int)

In [269]:
var0_less.tolist()

[85, 78, 86, 106, 91, 105, 103, 97, 104, 111, 91, 103, 115]

In [270]:
type(var0_less)

numpy.ndarray

In [271]:
# pred = []
# for i in range(x_test.shape[0]):
#     pred.append(exponent(x_test[i], mean[0],mean[1], var0_less,var1_less, Prior0,Prior1))
# predicted = np.array(pred)
# accuracy_score(predicted,y_test)

In [272]:
pred= np.zeros(x_test.shape[0])
e =2.718
for i in range(x_test.shape[0]):
    f = x_test[i,:].tolist()
    get_class(f,_var0,_means0, account1, private_key1,cont_pred)
    A0,B0,C0,D0 = cont_pred.functions.a_setparameters1().call()
    B0,C0,D0 = int(B0/100000000000000000000000000000000000000000000),int(C0/100000000000000000000000000000000000000000000),int(D0/100000000000000000000000000000000000000000000)
    print(A0,B0,C0,D0)
#     print(A0/B0*pow(e, -(C0/D0))*pow(Prior0,2))
    get_class(f,_var1, _means1, account1, private_key1,cont_pred)
    A1,B1,C1,D1 = cont_pred.functions.a_setparameters1().call()
    B1,C1,D1 = int(B1/100000000000000000000000000000000000000000000),int(C1/100000000000000000000000000000000000000000000),int(D1/100000000000000000000000000000000000000000000)
    get_probability(A0,A1,B0,B1,C0,C1,D0,D1,Prior0,Prior1, account1,private_key1,cont_pred)
    print(A1,B1,C1,D1)
#     print(A1/B1*pow(e, -(C1/D1))*pow(Prior1,2))
    Y0,Y1 = cont_pred.functions.set_exponent().call()
    print(Y0,Y1)
    pred[i] = compare(Y0,Y1)

1 58228998 1024916240 58228998
1 4544697 177972237 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 11436266619566433153361619277786012444620137048853897029636661391762432
1 58228998 1109811493 58228998
1 4544697 80576408 4544697
1083223828288207186161279388501620055954945501779462185836568427043200 27789202766730090982997186650372046401673662927340438333380427032154520000
1 58228998 1144270774 58228998
1 4544697 80740530 4544697
1083223828288207186161279388501620055954945501779462185836568427043200 48335231530126732829470700836276878946432579512192370775493306670746624000
1 58228998 325375943 58228998
1 4544697 129200454 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 8680888506487985777356893513757171923278222978100621820250831818736280
1 58228998 602341823 58228998
1 4544697 105966902 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 7683328995297722994225134738297710818686254238359926

1 58228998 957116268 58228998
1 4544697 46070098 4544697
1083223828288207186161279388501620055954945501779462185836568427043200 26977071828235076510299273496989112520636665947288425239007669761080320000
1 58228998 1162196789 58228998
1 4544697 126591152 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 8244435937744558709013881522417380651129301206306996170466998267523992
1 58228998 442193057 58228998
1 4544697 90593037 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 14229284327661822927663150982461900772424717326272083293495042918203672
1 58228998 926933519 58228998
1 4544697 87621393 4544697
1083223828288207186161279388501620055954945501779462185836568427043200000 13928234764356460747016103922833347905001061533929122270752750093456664
1 58228998 800504014 58228998
1 4544697 51686392 4544697
1083223828288207186161279388501620055954945501779462185836568427043200 29066179585453054710408077343760728653145498571167761832436

In [273]:
pred

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1.])

In [274]:
# pred = []
# for i in range(x_test.shape[0]):
#     pred.append(exponent(x_test[i], mean[0],mean[1], var[0],var[1], Prior0,Prior1))
# predicted = np.array(pred)
accuracy_score(pred,y_test)

0.8360655737704918

In [275]:
y_test

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1])

In [73]:
pca =PCA(n_components =12)
pca.fit(x_train,pca)
PC_train = (pca.transform(x_train)).astype(int)
PC_test = (pca.transform(x_test)).astype(int)
df_pc = pd.DataFrame(PC_train, columns=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'])
df_pc  ['target'] = y_train
df_mean = df_pc.groupby(['target']).mean().astype(int)
df_var = df_pc.groupby(['target']).var().astype(int)
mean = df_mean.values
var = df_var.values


In [249]:
pred = []
for i in range(x_test.shape[0]):
    pred.append(exponent(x_test[i], mean[0],mean[1], var[0],var[1], Prior0,Prior1))
predicted = np.array(pred)
accuracy_score(predicted,y_test)

1 6548608485616835397370613692558194360647172945665280 5.692141779315995e+23 6548608485616835397370613692558194360647172945665280
1 501820813590859976901325591517057831121236958105600 9.073313243552321e+23 501820813590859976901325591517057831121236958105600


/Users/Syed/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in long_scalars
  


OverflowError: int too large to convert to float

In [70]:
predicted = np.array(pred)
accuracy_score(predicted,y_test)

0.639344262295082

In [106]:
print(predicted)

[0 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0
 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1]


In [96]:
print(y_test)

[0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1
 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 0 0 1]
